In [1]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

# %load_ext tensorboard

In [2]:
train = pd.read_pickle('../data/train.p')
test = pd.read_pickle('../data/test.p')
signnames = pd.read_csv('../data/signnames.csv')

In [3]:
xTrain, yTrain = train['features'], train['labels']
xTest, yTest = test['features'], test['labels']

In [4]:
if yTrain.ndim == 1:
    yTrain = to_categorical(yTrain)
    
if yTest.ndim == 1:
    yTest = to_categorical(yTest)

In [5]:
inputShape = xTrain.shape[1:]
numClasses = yTrain.shape[1]

In [6]:
model = Sequential([
    Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', input_shape = inputShape),
    Flatten(),
    Dense(numClasses, activation = 'softmax')
])

model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
model.fit(xTrain, yTrain)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
flatten (Flatten)            (None, 57600)             0         
_________________________________________________________________
dense (Dense)                (None, 43)                2476843   
Total params: 2,478,635
Trainable params: 2,478,635
Non-trainable params: 0
_________________________________________________________________
1088/1088 [==============================] - 31s 28ms/step - loss: 11.0520 - accuracy: 0.7813


In [7]:
def getCNN_v1(aInputShape: list, aNumClasses: int) -> Sequential:
    return Sequential([
        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', input_shape = aInputShape),
        Flatten(),
        Dense(aNumClasses, activation = 'softmax')
    ])

def trainModel(aModel: Sequential, aXTrain, aYTrain, aParamsFit = {}):
    aModel.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
    
    logdir = os.path.join('logs', datetime.datetime.now().strftime('%y%m%d-%H%M%S'))
    tensorboardCallback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq = 1)
    
    aModel.fit(
        xTrain, 
        yTrain,
        batch_size = aParamsFit.get('batch_size', 128),
        epochs = aParamsFit.get('epochs', 5),
        verbose = aParamsFit.get('verbose', 1),
        validation_data = aParamsFit.get('validation_data', (xTrain, yTrain)),
        callbacks = [tensorboardCallback])
    
    return logdir

In [8]:
model = getCNN_v1(inputShape, numClasses)
logdir = trainModel(model, xTrain, yTrain)

Epoch 1/5
272/272 [==============================] - 27s 99ms/step - loss: 28.3521 - accuracy: 0.7494 - val_loss: 0.2432 - val_accuracy: 0.9424
Epoch 2/5
272/272 [==============================] - 27s 99ms/step - loss: 0.2025 - accuracy: 0.9543 - val_loss: 0.1150 - val_accuracy: 0.9716
Epoch 3/5
272/272 [==============================] - 27s 98ms/step - loss: 0.1222 - accuracy: 0.9722 - val_loss: 0.0752 - val_accuracy: 0.9802
Epoch 4/5
272/272 [==============================] - 26s 95ms/step - loss: 0.1140 - accuracy: 0.9733 - val_loss: 0.0896 - val_accuracy: 0.9809
Epoch 5/5
272/272 [==============================] - 28s 101ms/step - loss: 0.0926 - accuracy: 0.9785 - val_loss: 0.0953 - val_accuracy: 0.9755


In [9]:
# %tensorboard --logdir logs

In [10]:
def getCNN_v2(aInputShape: list, aNumClasses: int) -> Sequential:
    return Sequential([
        Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = aInputShape),
        MaxPool2D(),
        Dropout(0.5),
        
        Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', input_shape = aInputShape),
        MaxPool2D(),
        Dropout(0.3),
        
        Flatten(),
        
        Dense(1024, activation = 'relu'),
        Dropout(0.5),
        
        Dense(aNumClasses, activation = 'softmax')
    ])

getCNN_v2(inputShape, numClasses).summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2304)             

In [11]:
model = getCNN_v2(inputShape, numClasses)
paramsFit = {'epochs': 25}
trainModel(model, xTrain, yTrain, paramsFit)

Epoch 1/25
272/272 [==============================] - 40s 148ms/step - loss: 6.4246 - accuracy: 0.1979 - val_loss: 1.7975 - val_accuracy: 0.5207
Epoch 2/25
272/272 [==============================] - 36s 131ms/step - loss: 1.6474 - accuracy: 0.5143 - val_loss: 0.8450 - val_accuracy: 0.7867
Epoch 3/25
272/272 [==============================] - 35s 131ms/step - loss: 0.9990 - accuracy: 0.6944 - val_loss: 0.3737 - val_accuracy: 0.9068
Epoch 4/25
272/272 [==============================] - 35s 129ms/step - loss: 0.6936 - accuracy: 0.7869 - val_loss: 0.2286 - val_accuracy: 0.9480
Epoch 5/25
272/272 [==============================] - 35s 130ms/step - loss: 0.5386 - accuracy: 0.8314 - val_loss: 0.1503 - val_accuracy: 0.9638
Epoch 6/25
272/272 [==============================] - 35s 129ms/step - loss: 0.4480 - accuracy: 0.8604 - val_loss: 0.1291 - val_accuracy: 0.9703
Epoch 7/25
272/272 [==============================] - 35s 129ms/step - loss: 0.4061 - accuracy: 0.8767 - val_loss: 0.1068 - val_ac

'logs\\200401-185339'

In [12]:
# %tensorboard --logdir logs